<div class="admonition info">
    <p class="admonition-title">Info</p>
    <p>Click <a href="https://mybinder.org/v2/gh/oqtopus-team/quri-parts-oqtopus/HEAD?labpath=docs/usage/getting_started.ipynb" target="_blank">here</a> to run this notebook interactively on Binder.</p>
</div>

# Getting started

Welcome to QURI Parts OQTOPUS!

This page explains how to install QURI Parts OQTOPUS and use it with basic operations.

# Sampling on OQTOPUS Cloud

This section requires topics described in QURI Parts document ([Sampling simulation](https://quri-parts.qunasys.com/docs/tutorials/basics/sampler/)), so you need to read it before this section.

In QURI Parts document ([Sampling simulation](https://quri-parts.qunasys.com/docs/tutorials/basics/sampler/)), it is described how to estimate expectation value of operators using sampling measurements on a quantum circuit simulator. Since QURI Parts is designed to be platform independent, you can execute almost the same code on a real quantum computer.

## Prerequisite

**QURI Parts OQTOPUS** is a library that allows QURI Parts users to run quantum computers using the User API of OQTOPUS Cloud.

- **Provides a Backend for QURI Parts**: QURI Parts users can execute quantum programs on quantum computers in OQTOPUS Cloud using the backend provided by QURI Parts OQTOPUS.
- **Utilizes the User API of OQTOPUS Cloud**: QURI Parts OQTOPUS executes quantum programs via the User API of OQTOPUS Cloud and handles communication with the cloud. This allows users to run quantum computers without having to be aware of the communication protocols of OQTOPUS Cloud.

The User API of OQTOPUS Cloud is a REST interface, and post requests of jobs include OpenQASM 3 format quantum circuits.
However, this API currently supports only a subset of OpenQASM 3, such as basic gates and measurements.

In order to use OQTOPUS Cloud, you need to sign up for OQTOPUS Cloud. Please contact OQTOPUS Cloud operator for information on how to sign up. In this section, instead, we perform quantum circuits on the OQTOPUS Cloud.

You can install `quri-parts-oqtopus` as follows:

In [ ]:
!pip install quri-parts-oqtopus

## Prepare ``~/.oqtopus`` configuration file

Create a configuration file in path ``~/.oqtopus``.
Replace ``<base URL>`` and ``<API token>`` with your settings for OQTOPUS Cloud server and execute the following code:

In [1]:
from pathlib import Path

config = """[default]
url=<base URL>
api_token=<API token>
"""
Path("~/.oqtopus").expanduser().write_text(config)
print("Configuration saved.")

Configuration saved.


## Prepare a circuit

As a preparation, we create a circuit to be sampled:

In [2]:
from quri_parts.circuit import QuantumCircuit

circuit = QuantumCircuit(2)
circuit.add_H_gate(0)
circuit.add_CNOT_gate(0, 1)

## OqtopusSamplingBackend

In order to use a real device, you need to create a `OqtopusSamplingBackend` object. 

In [3]:
from quri_parts_oqtopus.backend import OqtopusSamplingBackend

# Create a SamplingBackend with the device
backend = OqtopusSamplingBackend()

The `OqtopusSamplingBackend` has `sample()` function, which returns a `OqtopusSamplingJob` object, and you can extract a result of the sampling job.
Please replace `device_id` with the one used in your system.

In [4]:
job = backend.sample(
    circuit,
    device_id="Kawasaki",
    shots=1000,
)

result = job.result()
print(result.counts)

Counter({0: 503, 3: 497})


## Sampling with transpiler

Specify information about the transpiler in the `transpiler_info` parameter of the sample function as a `dict`.
Set the name of transpiler library in `transpiler_lib` and the options to pass to the transpiler in `transpiler_options`.

For example, to apply Qiskit's transpiler with `optimization_level=2`, specify it as follows:

In [5]:
job = backend.sample(
    circuit,
    device_id="Kawasaki",
    shots=1000,
    transpiler_info={
        "transpiler_lib": "qiskit",
        "transpiler_options": {"optimization_level": 2},
    },
)

result = job.result()
print(result.counts)

Counter({3: 502, 0: 498})


To disable the transpiler, set `transpiler_lib` to `None`.

In [6]:
import math

circuit = QuantumCircuit(2)
circuit.add_RZ_gate(0, math.pi / 2)
circuit.add_SqrtX_gate(0)
circuit.add_RZ_gate(0, math.pi / 2)
circuit.add_CNOT_gate(0, 1)

job = backend.sample(
    circuit,
    device_id="Kawasaki",
    shots=1000,
    transpiler_info={
        "transpiler_lib": None,
    },
)

result = job.result()
print(result.counts)

Counter({3: 512, 0: 488})


## Sampler

Instead of using the backend directly, you can create a `Sampler` from it.
See the [document](https://quri-parts.qunasys.com/docs/tutorials/basics/sampler/) in QURI Parts document for more details.

# Estimation on OQTOPUS Cloud

## OqtopusEstimationBackend

In order to use a real device, you need to create a `OqtopusEstimationBackend` object. 

In [7]:
from quri_parts_oqtopus.backend import OqtopusEstimationBackend

# Create a OqtopusEstimationBackend with the device
backend = OqtopusEstimationBackend()

In [8]:
from quri_parts.circuit import QuantumCircuit
from quri_parts.core.operator import Operator, pauli_label

circuit = QuantumCircuit(2)
circuit.add_H_gate(0)
circuit.add_CNOT_gate(0, 1)

operator = Operator({
    pauli_label("X0 X1"): 1.0,
    pauli_label("Z0 Z1"): 1.0,
})

job = backend.estimate(
    circuit,
    operator=operator,
    device_id="Kawasaki",
    shots=10000,
)

result = job.result()
print(result.exp_value)

2.0


# QURI Parts OQTOPUS Examples

If you would like to view more examples, please refer to the following URL.
[https://github.com/oqtopus-team/quri-parts-oqtopus/tree/main/examples](https://github.com/oqtopus-team/quri-parts-oqtopus/tree/main/examples)